In [21]:
import requests
import pandas as pd
from datetime import datetime

In [22]:
SERVERS = [
    {"name": "Server1", "url": "http://127.0.0.1:8001", "type": "temperature"},
    {"name": "Server2", "url": "http://127.0.0.1:8002", "type": "temperature"},
    {"name": "PDU1", "url": "http://127.0.0.1:9001", "type": "power"}
]

# Redfish のエンドポイント設定
ENDPOINTS = {
    "temperature": "/redfish/v1/Chassis/1U/Thermal",
    "power": "/redfish/v1/Chassis/0/Power"
}

# 取得開始時刻（ファイル名用）
START_TIME = datetime.now().strftime("%Y%m%d_%H%M%S")

# 出力CSVファイル名
CSV_FILE = f"server_pdu_data_{START_TIME}.csv"

In [23]:
def fetch_data_with_timeout(server):
    """APIリクエストを実行し、タイムアウトやエラーを適切に処理"""
    url = f"{server['url']}{ENDPOINTS[server['type']]}"
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    try:
        response = requests.get(url, verify=False, timeout=5)  # SSL無視、タイムアウト5秒
        response.raise_for_status()
        return response.json(), url  # 成功時はデータとURLを返す

    except requests.exceptions.RequestException as e:
        print(f"❌ Error accessing {url}: {e}")
        return [{
            "Timestamp": timestamp,
            "Device": server["name"],
            "IP": server["url"],  # ここを追加
            "URL": url,  # URLも保持
            "Type": "Error",
            "Name": "N/A",  # エラー時のデータを統一
            "Value": "N/A",
            "Unit": "N/A",
            "Error Message": str(e)  # エラーメッセージを記録
        }], url


In [24]:
def fetch_data_with_timeout(server):
    """APIリクエストを実行し、タイムアウトやエラーを適切に処理"""
    url = f"{server['url']}{ENDPOINTS[server['type']]}"
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    try:
        response = requests.get(url, verify=False, timeout=5)  # SSL無視、タイムアウト5秒
        response.raise_for_status()
        return response.json(), url  # 取得成功時はデータとURLを返す

    except requests.exceptions.RequestException as e:
        print(f"❌ Error accessing {url}: {e}")
        return [{
            "Timestamp": timestamp,
            "Device": server["name"],
            "IP": server["url"],
            "URL": url,  # URLを追加
            "Type": "Error",
            "Name": None,
            "Value": None,
            "Unit": None,
            "Error Message": str(e)  # エラーメッセージを記録
        }], url

In [25]:
def extract_cpu_temperature(data, device_name, url):
    """Redfish APIのレスポンスからCPUの温度データを取得"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    results = []

    for sensor in data.get("Temperatures", []):
        if "CPU" in sensor.get("Name", ""):
            results.append({
                "Timestamp": timestamp,
                "Device": device_name,
                "URL": url,  # 追加
                "Type": "Temperature",
                "Name": sensor["Name"],
                "Value": sensor["ReadingCelsius"],
                "Unit": "Celsius"
            })

    return results

In [26]:
def extract_pdu_power(data, device_name, url):
    """Redfish APIのレスポンスからPDUの消費電力を取得"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    results = []

    for power_ctrl in data.get("PowerControl", []):
        results.append({
            "Timestamp": timestamp,
            "Device": device_name,
            "URL": url,  # 追加
            "Type": "Power",
            "Name": power_ctrl["Name"],
            "Value": power_ctrl["PowerConsumedWatts"],
            "Unit": "Watts"
        })

    return results

In [27]:
def fetch_and_display_data():
    """全サーバーと PDU からデータを取得し、DataFrame で表示"""
    all_data = []

    for server in SERVERS:
        print(f"\nFetching data from {server['name']} ({server['url']}) ...")
        
        data, url = fetch_data_with_timeout(server)
        
        # もし data がエラー情報を含んでいる場合、そのまま追加
        if isinstance(data, list) and data and data[0].get("Type") == "Error":
            all_data.extend(data)
            continue  # エラーならスキップ

        # データ取得が成功した場合、適切な処理関数を呼ぶ
        if server["type"] == "temperature":
            all_data.extend(extract_cpu_temperature(data, server["name"], url))
        elif server["type"] == "power":
            all_data.extend(extract_pdu_power(data, server["name"], url))

    if not all_data:
        print("⚠ No data found.")
        return pd.DataFrame()  # 空の DataFrame を返す

    df = pd.DataFrame(all_data)
    
    # CSVに保存
    df.to_csv(CSV_FILE, index=False)
    
    print(f"\n✅ Data saved to {CSV_FILE}")
    
    return df


In [28]:
df = fetch_and_display_data()
df


Fetching data from Server1 (http://127.0.0.1:8001) ...

Fetching data from Server2 (http://127.0.0.1:8002) ...

Fetching data from PDU1 (http://127.0.0.1:9001) ...
❌ Error accessing http://127.0.0.1:9001/redfish/v1/Chassis/0/Power: HTTPConnectionPool(host='127.0.0.1', port=9001): Read timed out. (read timeout=5)

✅ Data saved to server_pdu_data_20250211_091839.csv


,Timestamp,Device,URL,Type,Name,Value,Unit,IP,Error Message
0,2025-02-11 09:18:39,Server1,http://127.0.0.1:8001/redfish/v1/Chassis/1U/Th...,Temperature,CPU1 Temp,111.0,Celsius,NaN,NaN
1,2025-02-11 09:18:39,Server1,http://127.0.0.1:8001/redfish/v1/Chassis/1U/Th...,Temperature,CPU2 Temp,112.0,Celsius,NaN,NaN
2,2025-02-11 09:18:39,Server2,http://127.0.0.1:8002/redfish/v1/Chassis/1U/Th...,Temperature,CPU1 Temp,221.0,Celsius,NaN,NaN
3,2025-02-11 09:18:39,Server2,http://127.0.0.1:8002/redfish/v1/Chassis/1U/Th...,Temperature,CPU2 Temp,222.0,Celsius,NaN,NaN
4,2025-02-11 09:18:39,PDU1,http://127.0.0.1:9001/redfish/v1/Chassis/0/Power,Error,None,NaN,None,http://127.0.0.1:9001,"HTTPConnectionPool(host='127.0.0.1', port=9001..."
